In [7]:
from PIL import Image
import cv2 as cv
import os
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
from ultralytics import YOLO

sys.path.append(os.getenv('root', 0))
from utils.plot_wrapper import *
from utils.fix_resolutions import *
from utils import cropping

In [8]:
load_dotenv()
root = os.getenv("root", ".")

In [9]:
flank_txt = os.path.join(root, "data", "flank.txt")
output_path = os.path.join(root, "split_data")

In [10]:
with open(flank_txt, "r") as f:
    lines = f.readlines()

In [11]:
lines[0].split(",")[0]

'C:/Users/arnav/OneDrive/Desktop/New folder/Tiger-Identification-Project\\data\\K 1 M\\IMG_0011.JPG'

In [12]:
for line in tqdm(lines):
    line = line.strip()
    line = line.replace("'", "")
    if not line:
        continue

    path, flank = line.split(",")
    new_path = path.replace("data", "split_data\\unclean\\" + flank)

    os.makedirs(os.path.dirname(new_path), exist_ok=True)

    img = cv.imread(path)
    cv.imwrite(new_path, img)

  0%|          | 0/823 [00:00<?, ?it/s]

In [14]:
model = "yolov8n.pt"
model = cropping.load_yolo_model(model)

for line in tqdm(lines):
    line = line.strip()
    line = line.replace("'", "")
    if not line:
        continue

    path, flank = line.split(",")
    new_path = path.replace("data", "split_data\\clean\\" + flank)

    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    img = cv.imread(path)
    cleaned_img = cropping.crop_image(img, model, padding=0.05)

    cv.imwrite(new_path, cleaned_img)


  0%|          | 0/823 [00:00<?, ?it/s]

In [ ]:
resolutions = {}

for folder in os.listdir(os.path.join(output_path, "clean")):
    for tiger_name in os.listdir(os.path.join(output_path, "clean", folder)):
        for img in os.listdir(os.path.join(output_path, "clean", folder, tiger_name)):
            img_path = os.path.join(output_path, "clean", folder, tiger_name, img)
            image = cv.imread(img_path)
            if image is not None:
                res = (image.shape[1], image.shape[0])  # (width, height)
                resolutions[res] = resolutions.get(res, 0) + 1

print(resolutions)

{(1277, 679): 1, (1133, 569): 1, (1197, 597): 1, (1508, 820): 1, (814, 973): 1, (1661, 864): 1, (1448, 818): 1, (1370, 705): 1, (1011, 536): 1, (571, 886): 1, (1030, 546): 1, (1056, 825): 1, (740, 629): 1, (965, 679): 1, (671, 363): 1, (840, 433): 1, (1152, 570): 1, (653, 390): 1, (1084, 636): 1, (916, 519): 1, (938, 521): 1, (1041, 505): 1, (1003, 504): 1, (1056, 532): 1, (871, 580): 1, (247, 688): 1, (860, 674): 1, (860, 895): 1, (725, 954): 1, (1202, 864): 1, (886, 825): 1, (724, 934): 1, (853, 448): 1, (882, 444): 1, (829, 662): 2, (860, 524): 1, (1154, 828): 1, (841, 561): 1, (1569, 745): 1, (1316, 1072): 1, (1265, 908): 1, (1514, 1187): 1, (1923, 1127): 1, (1598, 1165): 1, (2048, 1536): 8, (732, 565): 1, (832, 478): 1, (868, 476): 1, (2048, 1127): 1, (1987, 1086): 1, (2048, 1500): 1, (1148, 848): 1, (1318, 909): 1, (1163, 711): 1, (1013, 732): 1, (872, 497): 1, (494, 951): 1, (950, 534): 1, (1005, 536): 1, (703, 398): 1, (901, 644): 1, (1054, 707): 1, (808, 628): 1, (708, 794): 1

In [15]:
target_size = (600, 300)

for folder in os.listdir(os.path.join(output_path, "clean")):
    for tiger_name in os.listdir(os.path.join(output_path, "clean", folder)):
        for img in os.listdir(os.path.join(output_path, "clean", folder, tiger_name)):
            os.makedirs(os.path.join(output_path, "resized", folder, tiger_name), exist_ok=True)
            img_path = os.path.join(output_path, "clean", folder, tiger_name, img)
            new_path = os.path.join(output_path, "resized", folder, tiger_name, img)
            image = cv.imread(img_path)
            if image is not None:
                resized_img = cv.resize(image, target_size)
                cv.imwrite(new_path, resized_img)